In [9]:
import pandas as pd
import numpy as np

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

In [129]:
data = pd.read_csv('data/combined_cleaned_data2/combined_with_fe.csv')

C:\Users\Admin\AppData\Local\Temp\ipykernel_13388\3770877507.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/combined_cleaned_data2/combined_with_fe.csv')


In [132]:
data.columns

Index(['reviewText', 'review_clean', 'sentiment'], dtype='object')

In [131]:
data = data[['reviewText', 'review_clean', 'sentiment']]

In [97]:
import sys
from nltk.tokenize import word_tokenize

In [133]:
corpus = data['reviewText'].tolist()

In [134]:
data.dtypes

reviewText      object
review_clean    object
sentiment       object
dtype: object

In [135]:
type(corpus[2])

str

In [136]:
len(corpus)

73624

In [138]:
def word_tokenize(sentence):
  return sentence.split()

In [139]:
data['tokenized'] = data['reviewText'].astype(str).apply(word_tokenize)

In [142]:
data['token_count'] = ""

In [143]:
for i in range(len(data['tokenized'])):
    data['token_count'][i] = len(data['tokenized'][i])

In [144]:
data

,reviewText,review_clean,sentiment,tokenized,token_count
0,"It diffuses a very mild light perfume, just wh...",wear mild perfume nice strong lotion day perfu...,Positive,"[It, diffuses, a, very, mild, light, perfume,,...",42
1,All time favorite!!!! Wish they still carried ...,time still favorite carry wish,Positive,"[All, time, favorite!!!!, Wish, they, still, c...",8
2,One of my wife's favorites.,one favorites wife's,Positive,"[One, of, my, wife's, favorites.]",5
3,If you have body acne this product is a must. ...,heal help within prevent already first wash mu...,Positive,"[If, you, have, body, acne, this, product, is,...",43
4,I really is what I expected,expect really,Positive,"[I, really, is, what, I, expected]",6
...,...,...,...,...,...
73619,When I heard about Rail Simulator I was happy ...,think guide go acela maybe i've realize star t...,Negative,"[When, I, heard, about, Rail, Simulator, I, wa...",1570
73620,very good,good,Positive,"[very, good]",2
73621,does a great job at a great price. Good quali...,great quality price good job,Positive,"[does, a, great, job, at, a, great, price., Go...",11
73622,Microsoft's point system doesn't really make a...,point else sense code point good make worked g...,Positive,"[Microsoft's, point, system, doesn't, really, ...",40


In [145]:
data.dtypes

reviewText      object
review_clean    object
sentiment       object
tokenized       object
token_count     object
dtype: object

In [146]:
data.token_count.astype(int) 

0          42
1           8
2           5
3          43
4           6
         ... 
73619    1570
73620       2
73621      11
73622      40
73623      69
Name: token_count, Length: 73624, dtype: int32

In [147]:
data = data[data['token_count'] < 512]

In [148]:
data.shape

(72643, 5)

In [149]:
print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

308970
54245


In [150]:
data.shape

(72643, 5)

In [151]:
import re

In [152]:
data.review_clean = data.review_clean.astype(str)

C:\Users\Admin\AppData\Local\Temp\ipykernel_13388\887699762.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.review_clean = data.review_clean.astype(str)


In [185]:
tokenizer = Tokenizer(nb_words=2000, lower=True,split=' ')
tokenizer.fit_on_texts(data['review_clean'].values)
# print(tokenizer.word_index)  # To see the dictionary

c:\Users\Admin\anaconda3\envs\capstone3\lib\site-packages\keras_preprocessing\text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [186]:
X = tokenizer.texts_to_sequences(data['review_clean'].values)
X = pad_sequences(X)

In [52]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(2500, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 719, 128)          320000    
                                                                 
 spatial_dropout1d_2 (Spatia  (None, 719, 128)         0         
 lDropout1D)                                                     
                                                                 
 lstm_3 (LSTM)               (None, 196)               254800    
                                                                 
 dense_3 (Dense)             (None, 2)                 394       
                                                                 
Total params: 575,194
Trainable params: 575,194
Non-trainable params: 0
_________________________________________________________________
None


In [187]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(48670, 178) (48670, 2)
(23973, 178) (23973, 2)


In [54]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
1542/1542 - 75s - loss: 0.3158 - accuracy: 0.8729 - 75s/epoch - 49ms/step
Epoch 2/7
1542/1542 - 74s - loss: 0.2627 - accuracy: 0.8929 - 74s/epoch - 48ms/step
Epoch 3/7
1542/1542 - 74s - loss: 0.2455 - accuracy: 0.9012 - 74s/epoch - 48ms/step
Epoch 4/7
1542/1542 - 78s - loss: 0.2311 - accuracy: 0.9081 - 78s/epoch - 50ms/step
Epoch 5/7
1542/1542 - 75s - loss: 0.2182 - accuracy: 0.9136 - 75s/epoch - 49ms/step
Epoch 6/7
1542/1542 - 76s - loss: 0.2073 - accuracy: 0.9193 - 76s/epoch - 49ms/step
Epoch 7/7
1542/1542 - 76s - loss: 0.1970 - accuracy: 0.9227 - 76s/epoch - 49ms/step


In [19]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

713/713 - 11s - loss: 0.2946 - accuracy: 0.8867 - 11s/epoch - 16ms/step
score: 0.29
acc: 0.89


In [55]:
from sklearn.metrics import confusion_matrix,classification_report

In [57]:
Y_pred = model.predict(X_test,batch_size = batch_size)
# predict_x=model.predict(X_test) 
Y_pred=np.argmax(Y_pred,axis=1)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

confusion matrix [[ 1605  2024]
 [  637 20030]]
              precision    recall  f1-score   support

           0       0.72      0.44      0.55      3629
           1       0.91      0.97      0.94     20667

    accuracy                           0.89     24296
   macro avg       0.81      0.71      0.74     24296
weighted avg       0.88      0.89      0.88     24296



In [155]:
from sklearn.utils import resample
from sklearn.utils import shuffle

In [188]:
# Separate majority and minority classes
data_majority = data[data['sentiment'] == 'Positive']
data_minority = data[data['sentiment'] == 'Negative']

bias = data_minority.shape[0]/data_majority.shape[0]
# lets split train/test data first then 
train = pd.concat([data_majority.sample(frac=0.8,random_state=200),
         data_minority.sample(frac=0.8,random_state=200)])
test = pd.concat([data_majority.drop(data_majority.sample(frac=0.8,random_state=200).index),
        data_minority.drop(data_minority.sample(frac=0.8,random_state=200).index)])

train = shuffle(train)
test = shuffle(test)

In [157]:
print('positive data in training:',(train.sentiment == 'Positive').sum())
print('negative data in training:',(train.sentiment == 'Negative').sum())
print('positive data in test:',(test.sentiment == 'Positive').sum())
print('negative data in test:',(test.sentiment == 'Negative').sum())

positive data in training: 49435
negative data in training: 8679
positive data in test: 12359
negative data in test: 2170


In [189]:
print('positive data in training:',(train.sentiment == 'Positive').sum())
print('negative data in training:',(train.sentiment == 'Negative').sum())
print('positive data in test:',(test.sentiment == 'Positive').sum())
print('negative data in test:',(test.sentiment == 'Negative').sum())

positive data in training: 49435
negative data in training: 8679
positive data in test: 12359
negative data in test: 2170


In [179]:
# Separate majority and minority classes in training data for upsampling 
data_majority = train[train['sentiment'] == 'Positive']
data_minority = train[train['sentiment'] == 'Negative']

print("majority class before upsample:",data_majority.shape)
print("minority class before upsample:",data_minority.shape)

# Upsample minority class
data_minority_upsampled = resample(data_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples= data_majority.shape[0],    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
data_upsampled = pd.concat([data_majority, data_minority_upsampled])
 
# Display new class counts
print("After upsampling\n",data_upsampled.sentiment.value_counts(),sep = "")

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['review_clean'].values) # training with whole data

X_train = tokenizer.texts_to_sequences(data_upsampled['review_clean'].values)
X_train = pad_sequences(X_train,maxlen=186)
Y_train = pd.get_dummies(data_upsampled['sentiment']).values
print('x_train shape:',X_train.shape)

X_test = tokenizer.texts_to_sequences(test['review_clean'].values)
X_test = pad_sequences(X_test,maxlen=186)
Y_test = pd.get_dummies(test['sentiment']).values
print("x_test shape", X_test.shape)

majority class before upsample: (49435, 5)
minority class before upsample: (8679, 5)
After upsampling
Positive    49435
Negative    49435
Name: sentiment, dtype: int64
x_train shape: (98870, 186)
x_test shape (14529, 186)


In [162]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(2500, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 186, 128)          320000    
                                                                 
 spatial_dropout1d_6 (Spatia  (None, 186, 128)         0         
 lDropout1D)                                                     
                                                                 
 lstm_7 (LSTM)               (None, 196)               254800    
                                                                 
 dense_7 (Dense)             (None, 2)                 394       
                                                                 
Total params: 575,194
Trainable params: 575,194
Non-trainable params: 0
_________________________________________________________________
None


In [69]:
import tensorflow as tf

In [70]:
tf.config.run_functions_eagerly(True)

In [163]:
batch_size = 128
# also adding weights
class_weights = {0: 1 ,
                1: 1/bias }
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 1,
          class_weight=class_weights)

Epoch 1/10


c:\Users\Admin\anaconda3\envs\capstone3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


773/773 [==============================] - 32s 41ms/step - loss: 0.9497 - accuracy: 0.6847
Epoch 2/10
773/773 [==============================] - 32s 41ms/step - loss: 0.8042 - accuracy: 0.7511
Epoch 3/10
773/773 [==============================] - 32s 42ms/step - loss: 0.7483 - accuracy: 0.7697
Epoch 4/10
773/773 [==============================] - 33s 42ms/step - loss: 0.6950 - accuracy: 0.7888
Epoch 5/10
773/773 [==============================] - 33s 42ms/step - loss: 0.6447 - accuracy: 0.8078
Epoch 6/10
773/773 [==============================] - 33s 42ms/step - loss: 0.5973 - accuracy: 0.8241
Epoch 7/10
773/773 [==============================] - 33s 42ms/step - loss: 0.5560 - accuracy: 0.8378
Epoch 8/10
773/773 [==============================] - 33s 43ms/step - loss: 0.5117 - accuracy: 0.8533
Epoch 9/10
773/773 [==============================] - 33s 43ms/step - loss: 0.4690 - accuracy: 0.8667
Epoch 10/10
773/773 [==============================] - 33s 43ms/step - loss: 0.4351 - accurac

In [180]:
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 1,
          class_weight=class_weights)

Epoch 1/10


c:\Users\Admin\anaconda3\envs\capstone3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


773/773 [==============================] - 33s 42ms/step - loss: 0.3933 - accuracy: 0.8926
Epoch 2/10
773/773 [==============================] - 33s 42ms/step - loss: 0.3603 - accuracy: 0.9008
Epoch 3/10
773/773 [==============================] - 33s 42ms/step - loss: 0.3349 - accuracy: 0.9090
Epoch 4/10
773/773 [==============================] - 33s 42ms/step - loss: 0.3130 - accuracy: 0.9148
Epoch 5/10
773/773 [==============================] - 33s 42ms/step - loss: 0.2997 - accuracy: 0.9200
Epoch 6/10
773/773 [==============================] - 33s 43ms/step - loss: 0.2807 - accuracy: 0.9246
Epoch 7/10
773/773 [==============================] - 33s 43ms/step - loss: 0.2688 - accuracy: 0.9284
Epoch 8/10
773/773 [==============================] - 33s 43ms/step - loss: 0.2536 - accuracy: 0.9320
Epoch 9/10
773/773 [==============================] - 33s 43ms/step - loss: 0.2414 - accuracy: 0.9344
Epoch 10/10
773/773 [==============================] - 34s 43ms/step - loss: 0.2349 - accurac

In [181]:
Y_pred = model.predict(X_test,batch_size = batch_size)
# predict_x=model.predict(X_test) 
Y_pred=np.argmax(Y_pred,axis=1)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

confusion matrix [[ 1089  1081]
 [  712 11647]]
              precision    recall  f1-score   support

           0       0.60      0.50      0.55      2170
           1       0.92      0.94      0.93     12359

    accuracy                           0.88     14529
   macro avg       0.76      0.72      0.74     14529
weighted avg       0.87      0.88      0.87     14529



In [182]:
model.save('model10.hdf5')

In [177]:
twt = ['this thing has no bad thing']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=186, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0 1399
    37 1576  123   37]]
1/1 - 0s - 28ms/epoch - 28ms/step
positi

In [89]:
Y = pd.get_dummies(data['sentiment']).values

In [90]:
Y

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [0, 1],
       [0, 1],
       [0, 1]], dtype=uint8)

In [183]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 0, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

c:\Users\Admin\anaconda3\envs\capstone3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


score: 0.59
acc: 0.88


In [184]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 0)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 94.95341614906833 %
neg_acc 51.886792452830186 %


In [190]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(2000, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 178, 128)          256000    
                                                                 
 spatial_dropout1d_7 (Spatia  (None, 178, 128)         0         
 lDropout1D)                                                     
                                                                 
 lstm_8 (LSTM)               (None, 196)               254800    
                                                                 
 dense_8 (Dense)             (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [191]:
# Separate majority and minority classes
data_majority = data[data['sentiment'] == 'Positive']
data_minority = data[data['sentiment'] == 'Negative']

bias = data_minority.shape[0]/data_majority.shape[0]
# lets split train/test data first then 
train = pd.concat([data_majority.sample(frac=0.8,random_state=200),
         data_minority.sample(frac=0.8,random_state=200)])
test = pd.concat([data_majority.drop(data_majority.sample(frac=0.8,random_state=200).index),
        data_minority.drop(data_minority.sample(frac=0.8,random_state=200).index)])

train = shuffle(train)
test = shuffle(test)

In [192]:
print('positive data in training:',(train.sentiment == 'Positive').sum())
print('negative data in training:',(train.sentiment == 'Negative').sum())
print('positive data in test:',(test.sentiment == 'Positive').sum())
print('negative data in test:',(test.sentiment == 'Negative').sum())

positive data in training: 49435
negative data in training: 8679
positive data in test: 12359
negative data in test: 2170


In [193]:
# Separate majority and minority classes in training data for upsampling 
data_majority = train[train['sentiment'] == 'Positive']
data_minority = train[train['sentiment'] == 'Negative']

print("majority class before upsample:",data_majority.shape)
print("minority class before upsample:",data_minority.shape)

# Upsample minority class
data_minority_upsampled = resample(data_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples= data_majority.shape[0],    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
data_upsampled = pd.concat([data_majority, data_minority_upsampled])
 
# Display new class counts
print("After upsampling\n",data_upsampled.sentiment.value_counts(),sep = "")

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['review_clean'].values) # training with whole data

X_train = tokenizer.texts_to_sequences(data_upsampled['review_clean'].values)
X_train = pad_sequences(X_train,maxlen=178)
Y_train = pd.get_dummies(data_upsampled['sentiment']).values
print('x_train shape:',X_train.shape)

X_test = tokenizer.texts_to_sequences(test['review_clean'].values)
X_test = pad_sequences(X_test,maxlen=178)
Y_test = pd.get_dummies(test['sentiment']).values
print("x_test shape", X_test.shape)

majority class before upsample: (49435, 5)
minority class before upsample: (8679, 5)
After upsampling
Positive    49435
Negative    49435
Name: sentiment, dtype: int64
x_train shape: (98870, 178)
x_test shape (14529, 178)


In [194]:
batch_size = 128
# also adding weights
class_weights = {0: 1 ,
                1: 1/bias }
history = model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 1,
          class_weight=class_weights)

Epoch 1/10


c:\Users\Admin\anaconda3\envs\capstone3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


773/773 [==============================] - 32s 41ms/step - loss: 0.9582 - accuracy: 0.6788
Epoch 2/10
773/773 [==============================] - 32s 42ms/step - loss: 0.8021 - accuracy: 0.7495
Epoch 3/10
773/773 [==============================] - 32s 42ms/step - loss: 0.7489 - accuracy: 0.7704
Epoch 4/10
773/773 [==============================] - 32s 42ms/step - loss: 0.7031 - accuracy: 0.7858
Epoch 5/10
773/773 [==============================] - 32s 42ms/step - loss: 0.6594 - accuracy: 0.8022
Epoch 6/10
773/773 [==============================] - 32s 42ms/step - loss: 0.6116 - accuracy: 0.8207
Epoch 7/10
773/773 [==============================] - 32s 42ms/step - loss: 0.5657 - accuracy: 0.8365
Epoch 8/10
773/773 [==============================] - 33s 42ms/step - loss: 0.5268 - accuracy: 0.8499
Epoch 9/10
773/773 [==============================] - 33s 43ms/step - loss: 0.4898 - accuracy: 0.8622
Epoch 10/10
773/773 [==============================] - 33s 43ms/step - loss: 0.4485 - accurac

In [195]:
batch_size = 128
# also adding weights
class_weights = {0: 1 ,
                1: 1/bias }
history = model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 1,
          class_weight=class_weights)

Epoch 1/10
773/773 [==============================] - 32s 41ms/step - loss: 0.4215 - accuracy: 0.8837
Epoch 2/10
773/773 [==============================] - 32s 42ms/step - loss: 0.3973 - accuracy: 0.8909
Epoch 3/10
773/773 [==============================] - 32s 42ms/step - loss: 0.3628 - accuracy: 0.9017
Epoch 4/10
773/773 [==============================] - 32s 42ms/step - loss: 0.3383 - accuracy: 0.9098
Epoch 5/10
773/773 [==============================] - 32s 42ms/step - loss: 0.3136 - accuracy: 0.9168
Epoch 6/10
773/773 [==============================] - 32s 42ms/step - loss: 0.2963 - accuracy: 0.9208
Epoch 7/10
773/773 [==============================] - 32s 42ms/step - loss: 0.2790 - accuracy: 0.9264
Epoch 8/10
773/773 [==============================] - 33s 42ms/step - loss: 0.2608 - accuracy: 0.9303
Epoch 9/10
773/773 [==============================] - 33s 43ms/step - loss: 0.2485 - accuracy: 0.9334
Epoch 10/10
773/773 [==============================] - 33s 43ms/step - loss: 0.238

In [196]:
Y_pred = model.predict(X_test,batch_size = batch_size)
# predict_x=model.predict(X_test) 
Y_pred=np.argmax(Y_pred,axis=1)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

confusion matrix [[ 1018  1152]
 [  630 11729]]
              precision    recall  f1-score   support

           0       0.62      0.47      0.53      2170
           1       0.91      0.95      0.93     12359

    accuracy                           0.88     14529
   macro avg       0.76      0.71      0.73     14529
weighted avg       0.87      0.88      0.87     14529



In [197]:
model.save('model11.hdf5')

In [211]:
twt = ['It cools the room very quickly and produces very smooth noise']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=178, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0  733  686  431  277  561  344 1012]]
1/1 - 0s - 26ms/epoch - 26ms/step
positive


In [203]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 0, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.62
acc: 0.88


In [204]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 0)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 96.18677042801556 %
neg_acc 49.76744186046512 %
